# Amélioration de DialogueGCN pour un Contexte Dynamique

## Problème  
DialogueGCN a une modélisation statique du contexte, ce qui limite sa capacité à capturer les nuances des émotions dans les conversations, notamment pour les interventions courtes ("okay", "oui", etc.).

## Solution  
Intégrer un mécanisme d’attention temporelle adaptative et un renforcement contextuel pour améliorer la flexibilité du modèle.

---

## 1. Parties du Code à Modifier

### (A) Mécanisme d’Attention Statique → Dynamique  
- **Fichier** : `models.py`  
- **Lignes concernées** : 516–616 (MaskedEdgeAttention)  
- **Méthodes** : `forward()` (l. 538) - Calcul des scores d’attention fixes.  
  
#### **Modification proposée** :  
```python
class AdaptiveTemporalAttention(nn.Module):
    def __init__(self, input_dim, max_seq_len, no_cuda):
        super().__init__()
        self.time_attn = nn.MultiheadAttention(input_dim, num_heads=4)  # Attention dynamique
        self.pos_encoder = PositionalEncoding(input_dim)  # Ajoute la temporalité

    def forward(self, M, lengths, edge_ind):
        M = self.pos_encoder(M)  # (seq_len, batch, D_e)
        attn_output, _ = self.time_attn(M, M, M)  # Recalcule les poids à chaque pas
        scores = self.compute_adaptive_scores(attn_output, edge_ind)  # Nouvelle méthode
        return scores
```
#### **Avantages** :
✅ Adaptation en temps réel aux changements conversationnels.  
✅ Remplace l’ancienne attention par une version basée sur Transformer.  

---

### (B) Fenêtres Temporelles Fixes → Adaptatives  
- **Fichier** : `models.py`  
- **Lignes concernées** : 619–720 (batch_graphify et edge_perms)  
- **Méthodes** : `edge_perms()` (l. 619) - Définit des fenêtres fixes (`window_past`, `window_future`).  
  
#### **Modification proposée** :  
```python
def dynamic_edge_perms(l, current_utterance, hidden_state, max_window=10):
    """ Prédit la fenêtre optimale en fonction du contexte. """
    window_size = self.window_predictor(current_utterance, hidden_state).clamp(1, max_window)
    return custom_edge_perms(l, window_size)
```
#### **Intégration** :  
- Remplacer `edge_perms` par `dynamic_edge_perms` dans `batch_graphify`.
- Ajouter un prédicteur de fenêtre (MLP) :  
```python
self.window_predictor = nn.Sequential(
    nn.Linear(D_e + D_h, 32),
    nn.ReLU(),
    nn.Linear(32, 1)
)
```

---

### (C) Renforcement Contextuel pour Mots Courts  
- **Fichier** : `models.py`  
- **Lignes concernées** : 723–875 (DialogueGCNModel)  
- **Méthodes** : `forward()` (l. 832) - Traitement des features sans adaptation contextuelle.  
  
#### **Modification proposée** :  
```python
class ContextEnhancer(nn.Module):
    def __init__(self, D_e):
        super().__init__()
        self.enhancer = nn.Sequential(
            nn.Linear(D_e * 2, D_e),  # Concatène utterance + contexte
            nn.ReLU(),
            nn.Linear(D_e, D_e))

    def forward(self, utterance, context):
        if utterance.size(1) < 3:  # Si l'intervention est courte
            return self.enhancer(torch.cat([utterance, context], dim=-1))
        return utterance
```
#### **Utilisation** :
Insérer dans `DialogueGCNModel.forward()` avant la classification :
```python
enhanced_features = self.context_enhancer(emotions, g_hist)
```

---

## 2. Schéma d’Intégration  
```
DialogueGCN (original)
│
├── (1) AdaptiveTemporalAttention (remplace MaskedEdgeAttention)
│    └── Calcule des scores d’attention dynamiques
│
├── (2) dynamic_edge_perms (remplace edge_perms)
│    └── Prédit la fenêtre temporelle optimale
│
└── (3) ContextEnhancer
     └── Renforce les mots courts avec le contexte global
```

---

## 3. Bénéfices Attendus  
| Composant                 | Amélioration                     | Impact                                      |
|---------------------------|---------------------------------|--------------------------------------------|
| AdaptiveTemporalAttention | Attention dynamique             | Meilleure capture des dépendances longues |
| dynamic_edge_perms        | Fenêtres adaptatives           | Réduction du biais des fenêtres fixes    |
| ContextEnhancer          | Contexte pour mots courts      | Analyse améliorée pour "okay", "oui"    |

---

## 4. Validation Expérimentale  

✅ **Métriques à surveiller** :  
- Accuracy sur les interventions courtes.  
- F1-score pour les émotions ambiguës.  
- Temps d’entraînement (vérifier l’overhead du Transformer).  

✅ **Jeu de test** :  
- Conversations avec nombreuses interventions courtes (**DAIC-WOZ, MELD**).  

---

## 5. Résumé des Fichiers Modifiés  

| Fichier    | Lignes   | Modification |
|------------|---------|--------------|
| `models.py` | 516–616 | Nouvelle classe `AdaptiveTemporalAttention` |
| `models.py` | 619–720 | Remplacement de `edge_perms` |
| `models.py` | 832 (forward) | Ajout de `ContextEnhancer` |

---

## 6. Étapes Suivantes  
1. Implémenter les nouvelles classes.  
2. Benchmark sur **MELD/IEMOCAP**.  
3. Fine-tuning pour équilibrer performance/complexité.  
 
